In [1]:
%run lib.py

---

In [2]:
import textwrap
from copy import deepcopy

from ete3 import AttrFace, TextFace, Tree, TreeStyle, add_face_to_node, NodeStyle, RectFace, CircleFace

from anytree.importer import JsonImporter
from pipe import reverse, as_list, as_set

import visual
from taxonomy import print_tree


In [3]:
with open('ds_taxonomy.json', 'r') as f:
    tax = JsonImporter().read(f)

In [4]:
lifting_pars_df = pd.read_excel('lifting_results.xlsx', index_col=[0,1])
lifted_trees = joblib.load( 'data/lifted_trees.pkl')

In [53]:
tree = reload(visual).compress_pgfs_result(lifted_trees[6], 0.2)

In [54]:
newick_tax = reload(visual).pgfs_result_to_newick(tree)

t = Tree(newick_tax, format=1, quoted_node_names=False)

t.show(tree_style=reload(visual).get_tree_style())

---
Batch

In [60]:
def get_head_subjects(root):
    heads_and_offshoots = root.H
    offshoots = set()
    for n in heads_and_offshoots:
        node = anytree.find(root, filter_=lambda x: x.name == n)
        if node and node.is_leaf:
            offshoots.add(n)
    head_subjects = heads_and_offshoots - offshoots
    return head_subjects

In [61]:
reload(visual)

U_THRESH = 0.2
    
for i, lift_result in enumerate(lifted_trees):
    
    tree = visual.compress_pgfs_result(lift_result, U_THRESH)
    newick_tree = visual.pgfs_result_to_newick(tree)
    
    t = Tree(newick_tree, format=1, quoted_node_names=False)
    
    if len(get_head_subjects(tree)) == 0:
        fname = f'images/cluster_lift_tree_{i:02d}_no_heads.pdf'
    else:
        fname = f'images/cluster_lift_tree_{i:02d}.pdf'
    
    t.render(fname, tree_style=visual.get_tree_style())

---